<h1> <center> Logistic regression </center> </h1>

# 1. Model specification

Logistic regression corresponding to the following binary classification model
$$
p(y=1|\vec{x},\vec{w})=Ber(y|sigm(\vec{w}^T\vec{x})
$$
where 
$$
sigm(\eta)=\frac{1}{1+exp(-\eta)}
$$ 
is the logistic function.For multi-class logistic regression, the model becomes
$$
p(y=c|\vec{x},\mathbf{W})=\frac{exp(\vec{w}_c^T\vec{x})}{\sum_{c'=1}^C exp(\vec{w}_{c'}^T\vec{x})}
$$
The negative log-likelihood function of Logistic regression is given as 
\begin{align}
p(D|\vec{w}) &=-\sum_{i=1}^N log \left[ \mu_i^{\mathbb{1}(y_i=1)} \left(1-\mu_i)\right)^{\mathbb{1}(y_i=0)} \right] \\
             &=-\sum_{i=1}^N \left[\mathbb{1}(y_i=1) log \mu_i+\mathbb{1}(y_i=0) log(1-\mu_i) \right ]
\end{align}
where $\mu_i=sigm(\vec{w}^T\vec{x}_i)$. The negative log-likelihood function is also called the **cross-entropy** loss function sometimes.

# 2. MLE solution
For logistic regression, we can't get the closed form solution anymore. Instead, we need to use an optimization algorithm to compute the MLE solution.
The simplest algorithm for unconstrained optimization is **gradient descent**,also known as **steepest descent**. For logistic regression, the negative log-likelihood function is as follows
\begin{align*}
NLL(\vec{w}) &=-\sum_{i=1}^N \left[\mathbb{1}(y_i=1) log \mu_i+\mathbb{1}(y_i=0) log(1-\mu_i) \right] \\
\mu_i &=sigm(\vec{w}^T\vec{x_i})
\end{align*}
Our task is to minimize the negative log-likelihood function respecting to $\vec{w}$. We can write the gradient as 
\begin{align}
\frac{\partial NLL(\vec{w})}{\partial \vec{w}} &=-\sum_{i=1}^N \lbrace \mathbb{1}(y_i=1) \frac{\partial (log\mu_i)}{\partial \vec{w}} +\mathbb{1}(y_i=0) \frac{\partial (log(1-\mu_i))}{\partial \vec{w}} \rbrace\\
                                               &=-\sum_{i=1}^N \lbrace \mathbb{1}(y_i=1) \frac{1}{\mu_i}\frac{\partial \mu_i}{\partial \vec{w}}+\mathbb{1}(y_i=0) \frac{-1}{1-\mu_i}\frac{\partial \mu_i}{\partial \vec{w}}\rbrace
\end{align}
The gradient of the logistic function are given by the following 
\begin{align}
g &=\frac{\partial \, \mu}{\partial \vec{w}}   \\
  &=\frac{\partial\, sigm(\vec{w}^T\vec{x})}{\partial \vec{w}} \\
  &=\mu(1-\mu)\vec{x}
\end{align}
We can get the overall gradient as
\begin{align}
\frac{\partial NLL(\vec{w})}{\partial \vec{w}} &=-\sum_{i=1}^N \lbrace \mathbb{1}(y_i=1)(1-\mu_i)\vec{x}+\mathbb{1}(y_i=0) (-\mu_i) \vec{x} \rbrace \\
 &=\sum_{i=1}^N(\mu_i-y_i)\vec{x}_i 
\end{align}
The steepest descent update formular is as follows
$$
\vec{w}_{k+1}=\vec{w}_{k}-\eta_k \left( \frac{\partial NLL(\vec{w})}{\partial \vec{w}} \right)_k
$$
where $\eta_k$ is the learning rate.

In [29]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(42)

class logistic_regression:
    def __init__(self,valid_freq=100):
        '''
        Initialize the model
        
        Parameters:
            valid_freq:
                validate frequence
        '''
        self.valid_freq=valid_freq
    
    def fit(self,X_train,y_train,X_valid=None,y_valid=None,max_iters=2000,batch_size=128,learning_rate=0.01):
        '''
        Fit a logistic model
        
        Parameters:
            X_train:array-like,shape(n_samples,n_features)
                train data
            y_train:array-like,shape(n_samples,)
                train target
            X_valid:array-like,shape(n_samples,n_features),optional, default None
                validate data
            y_valid:array-like,shape(n_samples),optional,default None
                validate target
            max_iters:int,optional, default 2000
                maximum iterations
            batch_size:int,optional,default 32
                batch size
            learning_rate:float,optional,default 0.01
                learning rate
        '''
        with tf.Graph().as_default():
            n_features=X_train.shape[1]
            n_classes=np.unique(y_train).size
            
            weight=tf.get_variable("weight",shape=[n_features,n_classes],
                                   initializer=tf.random_normal_initializer())
            bias=tf.get_variable("bias",shape=[n_classes],
                                 initializer=tf.constant_initializer(0.0))
            
            X_train=X_train.astype(np.float32)
            y_train=y_train.astype(np.int64)
            
            train_dataset=tf.data.Dataset.from_tensor_slices((X_train,y_train))
            train_dataset=train_dataset.batch(batch_size).repeat()
            
            valid_dataset=train_dataset  # valid_dataset must be existed no matter X_valid is None or not
            valid_flag= (X_valid is not None and y_valid is not None)
            if valid_flag:
                X_valid=X_valid.astype(np.float32)
                y_valid=y_valid.astype(np.int64)
                valid_dataset=tf.data.Dataset.from_tensor_slices((X_valid,y_valid))
                valid_dataset=valid_dataset.batch(batch_size).repeat()
            
            training=tf.placeholder(dtype=tf.bool)
            x,y_true=tf.cond(training,lambda:train_dataset.make_one_shot_iterator().get_next(),
                            lambda:valid_dataset.make_one_shot_iterator().get_next())
            
            y_pred=tf.matmul(x,weight)+bias  # y_pred with shape(n_samples,n_classes)
            y_most_like=tf.cast(tf.argmax(y_pred,axis=1),tf.int64)
            accuracy=tf.reduce_sum(tf.cast(tf.equal(y_true,y_most_like),tf.float32))/ batch_size
            loss=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)
            train_op=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
            
            init=tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
            with tf.Session() as sess:
                sess.run(init)
                for i in range(max_iters):
                    _=sess.run(train_op,feed_dict={training:True})
                    if i%self.valid_freq==0:
                        accu=sess.run(accuracy,feed_dict={training:False})
                        print("The accuracy on validate set of %d th iteration  is :%.4f " %(i,accu))
                        
mnist=tf.keras.datasets.mnist
(X_train,y_train),(X_valid,y_valid)=mnist.load_data()
X_train,X_valid=X_train/255.0,X_valid/55.0
X_train=X_train.reshape((X_train.shape[0],-1))
X_valid=X_valid.reshape((X_valid.shape[0],-1))

model=logistic_regression(valid_freq=1000)
model.fit(X_train,y_train,X_valid,y_valid,max_iters=10000,learning_rate=0.01)
        

The accuracy on validate set of 0 th iteration  is :0.1172 
The accuracy on validate set of 1000 th iteration  is :0.8672 
The accuracy on validate set of 2000 th iteration  is :0.8594 
The accuracy on validate set of 3000 th iteration  is :0.8828 
The accuracy on validate set of 4000 th iteration  is :0.8359 
The accuracy on validate set of 5000 th iteration  is :0.8438 
The accuracy on validate set of 6000 th iteration  is :0.8750 
The accuracy on validate set of 7000 th iteration  is :0.8047 
The accuracy on validate set of 8000 th iteration  is :0.8125 
The accuracy on validate set of 9000 th iteration  is :0.7500 
